In [57]:
import pandas as pd

solar_data = pd.read_csv("./historicData/Postcode data for small-scale installations - SGU-Solar.csv", index_col="Small Unit Installation Postcode")
solar_data.index.name = "postcode"
columns = solar_data.columns

print("create postcode list ...")
data_postcodes = list(solar_data.index)

print("saving postcode totals ...")
postcode_total = solar_data[["Installations Quantity Total"]]
postcode_total.columns = ["installations"]
postcode_total["installations"] = postcode_total["installations"].apply(lambda x: int(x.replace(",", "")))
postcode_total.to_csv("./solarData/postcode_total.csv")

print("calculate valid postcodes ...")
poa = pd.read_csv("./valid_postcodes.csv")
valid_postcodes = list(poa["postcode"])

postcodes = []
for postcode in data_postcodes:
    if postcode in valid_postcodes:
        postcodes.append(postcode)
pd.Series(postcodes).to_csv("./filtered_postcodes.csv")

solar_time_series = pd.DataFrame(index=solar_data.index)

for year in range(2001, 2021):
    print(f"calculating {year} time-series data ...")
    if year > 2018:
        data_url = "./historicData/Postcode data for small-scale installations - SGU-Solar.csv"
    else:
        data_url = f"./historicData/Postcode data for small-scale installations {year} - SGU-Solar.csv"
    temp_data = pd.read_csv(data_url, index_col="Small Unit Installation Postcode")    
    filtered_columns = []
    for column in temp_data.columns:
        if f"{year} - Installations Quantity" in column:
            filtered_columns.append(column)
    temp_data = temp_data[filtered_columns]
    for column in temp_data.columns:
        solar_time_series[column[:8]] = temp_data[column]

print("saving monthly installations ...")
monthly_installs = pd.DataFrame(solar_time_series.sum())
monthly_installs.columns = ["installations"]
monthly_installs["cumulative"] = monthly_installs["installations"].cumsum()
monthly_installs.to_csv("./solarData/monthly_installs.csv", index_label="month")

print("saving time-series data ...")
solar_time_series = solar_time_series.cumsum(axis=1)
solar_time_series.to_csv("./solarData/solar_timeseries.csv")

df1 = pd.read_csv("./solarData/solarPostcodeIncome2016house.csv", index_col="postcode")
df2 = pd.read_csv("./solarData/solarPostcodeIncome2016semi1.csv", index_col="postcode")
df3 = pd.read_csv("./solarData/solarPostcodeIncome2016semi2.csv", index_col="postcode")
solar_incomes = df1 + df2 + df3
solar_incomes.index = pd.Series(solar_incomes.index).apply(lambda x: x[:4])

df1 = pd.read_csv("./solarData/solarPostcodeTenure2016house.csv", index_col="postcode")
df2 = pd.read_csv("./solarData/solarPostcodeTenure2016semi1.csv", index_col="postcode")
df3 = pd.read_csv("./solarData/solarPostcodeTenure2016semi2.csv", index_col="postcode")
solar_tenure = df1 + df2 + df3
solar_tenure.index = pd.Series(solar_tenure.index).apply(lambda x: x[:4])

postcode_check = []
for postcode in solar_incomes.index:
    if int(postcode) in postcodes:
        postcode_check.append(postcode)
        
solar_incomes = solar_incomes.loc[postcode_check]
solar_incomes = solar_incomes[solar_incomes.columns[:-4]]
solar_incomes["nil"] = solar_incomes["Negative income"] + solar_incomes["Nil income"]
solar_incomes = solar_incomes[["nil"] + list(solar_incomes.columns[2:-1])]
solar_incomes.to_csv("./solarData/solar_incomes.csv")

solar_tenure = solar_tenure.loc[postcode_check]
solar_tenure["rented"] = solar_tenure["Rented"] + solar_tenure["Being occupied rent-free"] + solar_tenure["Being occupied under a life tenure scheme"]
solar_tenure = solar_tenure[list(solar_tenure.columns[:2]) + ["rented"]]
solar_tenure.columns = ["outright", "mortgage", "rent"]
solar_tenure.to_csv("./solarData/solar_tenure.csv")

dwellings2016 = pd.read_csv("./solarData/dwellings2016.csv")
new_index = []
for postcode in dwellings2016["postcode"]:
    new_index.append(int(postcode[:4]))
dwellings2016["postcode"] = new_index    
dwellings2016.set_index("postcode", inplace=True)
postcode_check = []
for postcode in dwellings2016.index:
    if postcode in postcodes:
        postcode_check.append(postcode)
dwellings2016 = dwellings2016.loc[postcode_check]
dwellings2016 = dwellings2016[["dwellings"]]

dwellings2011 = pd.read_csv("./solarData/dwellings2011.csv")
new_index = []
for postcode in dwellings2011["postcode"]:
    new_index.append(int(postcode[:4]))
dwellings2011["postcode"] = new_index    
dwellings2011.set_index("postcode", inplace=True)
postcode_check = []
for postcode in dwellings2011.index:
    if postcode in postcodes:
        postcode_check.append(postcode)
dwellings2011 = dwellings2011.loc[postcode_check]

households = []
for postcode in postcode_total.index:
    if postcode in postcodes:
        if postcode in dwellings2016.index:
            houses2016 = dwellings2016.loc[postcode][0]
            if postcode in dwellings2011.index:
                houses2011 = dwellings2011.loc[postcode][0]
                households.append(houses2016 + ((houses2016 / houses2011 - 1) * .8))
            else:
                households.append(houses2016)
        else:
            households.append("NaN")
    else:
        households.append("NaN")
postcode_total["households"] = households
install_rates = []
for postcode in postcode_total.index:
    installs = float(postcode_total.loc[postcode, "installations"])
    houses = float(postcode_total.loc[postcode, "households"])
    if (installs > 0) and (houses > 0):
        install_rates.append(installs / houses)
    else:
        install_rates.append("NaN")
postcode_total["install_rates"] = install_rates

# postcode_total["rate"] = postcode_total["installations"] / postcode_total["households"]
postcode_total["households"] = postcode_total["households"].astype("float")
postcode_total["install_rates"] = postcode_total["install_rates"].astype("float")
postcode_total.to_csv("./solarData/postcode_total.csv")

create postcode list ...
saving postcode totals ...
calculate valid postcodes ...
calculating 2001 time-series data ...
calculating 2002 time-series data ...
calculating 2003 time-series data ...
calculating 2004 time-series data ...
calculating 2005 time-series data ...
calculating 2006 time-series data ...
calculating 2007 time-series data ...
calculating 2008 time-series data ...
calculating 2009 time-series data ...
calculating 2010 time-series data ...
calculating 2011 time-series data ...
calculating 2012 time-series data ...


<ipython-input-57-c72248ea3ddf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postcode_total["installations"] = postcode_total["installations"].apply(lambda x: int(x.replace(",", "")))


calculating 2013 time-series data ...
calculating 2014 time-series data ...
calculating 2015 time-series data ...
calculating 2016 time-series data ...
calculating 2017 time-series data ...
calculating 2018 time-series data ...
calculating 2019 time-series data ...
calculating 2020 time-series data ...
saving monthly installations ...
saving time-series data ...


<ipython-input-57-c72248ea3ddf>:114: RuntimeWarning: invalid value encountered in long_scalars
  households.append(houses2016 + ((houses2016 / houses2011 - 1) * .8))
<ipython-input-57-c72248ea3ddf>:114: RuntimeWarning: divide by zero encountered in long_scalars
  households.append(houses2016 + ((houses2016 / houses2011 - 1) * .8))
<ipython-input-57-c72248ea3ddf>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postcode_total["households"] = households
<ipython-input-57-c72248ea3ddf>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [59]:
final_postcodes = postcode_total.dropna()
median_incomes = pd.read_csv("./solarData/medianIncomes.csv", index_col="postcode")
solar_exposure = pd.read_csv("./solarData/solar_exposure.csv", index_col="postcode")
solar_tenure["total"] = solar_tenure["outright"] + solar_tenure["mortgage"] + solar_tenure["rent"]
solar_tenure["owners"] = (solar_tenure["outright"] + solar_tenure["mortgage"]) / solar_tenure["total"]

incomes = []
owners = []
solarExposure = []

def convert_postcode(x):
    x = str(x)
    x = (4 - len(x)) * "0" + x
    return x

for postcode in final_postcodes.index:
    incomes.append(median_incomes.loc[postcode, "income"])
    owners.append(solar_tenure.loc[convert_postcode(postcode), "owners"])
    solarExposure.append(solar_exposure.loc[postcode, "solarExposure"])
final_postcodes["income"] = incomes
final_postcodes["owners"] = owners
final_postcodes["solarExposure"] = solarExposure
final_postcodes.columns = ["installations", "households", "installRate", "income", "ownership", "solar"]
final_postcodes.to_csv("./solarData/final_postcodes.csv")
final_postcodes

<ipython-input-59-c9730ca85dc2>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_postcodes["income"] = incomes
<ipython-input-59-c9730ca85dc2>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_postcodes["owners"] = owners
<ipython-input-59-c9730ca85dc2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,installations,households,installRate,income,ownership,solar
postcode,,,,,,
800,101,47.885714,2.109189,"$2,264",0.344828,20.2
810,3023,9252.022583,0.326739,"$2,107",0.613788,20.1
812,1637,6202.992462,0.263905,"$2,028",0.655329,20.1
820,980,4942.016353,0.198300,"$2,341",0.522384,20.1
822,774,6523.185906,0.118654,"$1,294",0.309911,20.7
...,...,...,...,...,...,...
7466,4,17.000000,0.235294,$612,1.000000,11.3
7467,71,1063.000000,0.066792,$764,0.745902,11.3
7468,30,446.033645,0.067260,"$1,312",0.650558,11.3
